In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet_v2 import preprocess_input 

import wandb # wandb登入 是為了將訓練過程中的結果上傳至雲端，方便我們查看訓練過程
model_log = {
    'dense_units': 1024,
    'dropout_rate': 0.4,
    'learning_rate': 0.0001
    }

model_name = "MobileNetV2_size16"
wandb.login()
wandb.init(
    project="100",
    name=model_name,
    config=model_log)

# from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Input , Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# 混淆矩陣套用列表
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score


In [ ]:
# 設定圖片大小
image_size = (224, 224)
# 設定批次大小
batch_size = 16
# 設定類別數量，根據你的資料集而定
num_classes = 18
# 設定訓練輪次
epochs = 100
# 設定儲存路徑h5檔
file_name = model_name

In [ ]:
# 使用ImageDataGenerator設定資料生成器
train_datagen = ImageDataGenerator(
    # rescale=1.0 / 255,
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    # # shear_range=0.2, # 浮點數。剪切強度（弧度的剪切角度）。1
    zoom_range=0.2, # 浮點數或元組。用於隨機縮放的範圍。2
    horizontal_flip=True, # 布林值。隨機水平翻轉輸入。3
    rotation_range=20, # 整數。用於隨機旋轉的度數範圍。4
    # # width_shift_range=0.2, #浮點數（總寬度的一部分）。用於隨機水平平移的範圍。5
    # # height_shift_range=0.2, # 浮點數（總高度的一部分）。用於隨機垂直平移的範圍。6
    channel_shift_range=35, # 浮點數。用於隨機通道平移的範圍。7  channel_shift_range: 對圖片的顏色通道進行隨機平移，可以增加顏色變化
    brightness_range=(0.8, 1.2), # 元組或列表。用於隨機亮度的範圍。8
)

# 使用ImageDataGenerator設定資料生成器
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
    )

In [ ]:
# 訓練集資料生成器
train_generator = train_datagen.flow_from_directory(
    "F:\工程師資料夾\MobileNetV3Large_K專用訓練_轉", # 你的照片樣本路徑 可至https://docs.google.com/spreadsheets/d/1dGP6clYpPmRioEUF7d0kHMFQjOeTYCjUANKakeGwfFo/edit#gid=254245868下載
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    # seed=42 # 亂數種子42  在這個系列中，一個超級電腦被設計出來來回答宇宙中所有問題的答案，而這個答案就是 “42”。
)

# 驗證集資料生成器
validation_generator = val_datagen.flow_from_directory(
    "F:\工程師資料夾\MobileNetV3Large_K專用訓練_轉",# 你的照片樣本路徑,可至https://docs.google.com/spreadsheets/d/1dGP6clYpPmRioEUF7d0kHMFQjOeTYCjUANKakeGwfFo/edit#gid=254245868下載
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

In [ ]:
# tensorboard_callback = TensorBoard(log_dir="logs", histogram_freq=1)

# 提前停止（Early Stopping）
early_stopping = EarlyStopping(
    monitor="val_loss", patience=30, restore_best_weights=True
)

# 設置 ModelCheckpoint 以保存最佳模型
model_checkpoint = ModelCheckpoint(
    f"path/{file_name}.h5",
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1,
)


In [ ]:
input_tensor = Input(shape=(224, 224, 3))

# 添加預訓練的 MobileNetV2 模型
base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=input_tensor)

# 凍結 MobileNetV2 層
for layer in base_model.layers[:]:
    layer.trainable = False

In [ ]:
# 全局平均池化層
x = GlobalAveragePooling2D()(base_model.output)

# 全連接層，使用 ReLU 激活函數
x = Dense(wandb.config.dense_units, activation="relu")(x)

# Dropout 層，防止過擬合
x = Dropout(wandb.config.dropout_rate)(x)

# 全連接層，使用 softmax 激活函數
output_tensor = Dense(len(train_generator.class_indices), activation="softmax")(x)

# 建立模型
model = Model(inputs=input_tensor, outputs=output_tensor)
model.compile(optimizer=Adam(lr=wandb.config.learning_rate), loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
wandb.config.update(model_log)
# wandb_callback  = wandb.keras.WandbCallback(log_weights=False, log_gradients=False)
model.summary()

In [ ]:
# # 訓練模型
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // batch_size,
#     callbacks=[tensorboard_callback, early_stopping, model_checkpoint],
# )

# # 訓練模型 完整訓練
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ model_checkpoint,wandb.keras.WandbCallback()],
)

In [ ]:
# for layer in model.layers:
    # print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

In [ ]:
# # 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
# %load_ext tensorboard
# # 啟動 Tensorboard
# %tensorboard --logdir log/directory

In [ ]:
# 以下是混淆矩陣
# 生成驗證集的預測
predictions = model.predict(validation_generator)
print("驗證",predictions)
#獲取預測的類別
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

# 獲取真實的類別
true_classes = validation_generator.classes
print("真實",true_classes)
# 創建混淆矩陣
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# 打印混淆矩陣
print("Confusion Matrix:")
print(conf_matrix)

# 生成分類報告
class_names = list(validation_generator.class_indices.keys())
class_report = classification_report(true_classes, predicted_classes, target_names=class_names)

# 打印分類報告
print("Classification Report:")
print(class_report)



In [ ]:
# 設定中文字體
sns.set(font='Microsoft YaHei')  # 使用中文字體（這裡使用的是思源黑體）

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('混淆矩陣')
plt.xlabel('預測值')
plt.ylabel('實際值')

# 儲存混淆矩陣的圖  (bbox_inches='tight' 調整圖片的邊界)
plt.savefig(f'path/{file_name}.png', bbox_inches='tight')
plt.show()

In [ ]:
# Calculate precision, recall, and F1-score for each class
precision_per_class = precision_score(true_classes, predicted_classes, average=None)
recall_per_class = recall_score(true_classes, predicted_classes, average=None)
f1_score_per_class = f1_score(true_classes, predicted_classes, average=None)

precision_per_class *=100
recall_per_class *=100 
f1_score_per_class *=100

# Create a DataFrame to display the metrics per class
metrics_per_class = pd.DataFrame({
    'Class': class_names,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_score_per_class
})

metrics_per_class = metrics_per_class.round(2)

print(metrics_per_class)